In [ ]:
import cv2
import face_recognition
import numpy as np
import pickle
import os

# Load Haar cascades
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
upperbody_cascade = cv2.CascadeClassifier('haarcascade_upperbody.xml')

# Load the trained KNN model
with open('models/trained_knn_model.clf', 'rb') as f:
    knn_clf = pickle.load(f)

# Initialize video capture
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale for Haar cascades
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    # Convert frame to RGB for face_recognition
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect face locations and encodings
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    # Initialize list for names
    names = []

    for face_encoding in face_encodings:
        # Use KNN model to find the best match
        closest_distances = knn_clf.kneighbors([face_encoding], n_neighbors=1)
        is_recognized = closest_distances[0][0][0] <= 0.5

        if is_recognized:
            name = knn_clf.predict([face_encoding])[0]
        else:
            name = "Unknown"

        names.append(name)


# Annotate recognized faces
for (top, right, bottom, left), name in zip(face_locations, names):
    # Draw rectangle around face
    cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
    # Label the face (shows "Unknown" if not matched)
    cv2.putText(frame, name, (left, bottom + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

# Detect upper bodies
upperbodies = upperbody_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

# Annotate upper bodies without detected faces (head down or not visible)
for (x, y, w, h) in upperbodies:
    # Check if this upper body overlaps with any detected face
    overlaps = False
    for (fx, fy, fw, fh) in faces:
        if (fx < x + w and fx + fw > x and fy < y + h and fy + fh > y):
            overlaps = True
            break
    if not overlaps:
        # Optionally, check if upper body is high in the frame (head down)
        # if y < frame.shape[0] // 2:  # Uncomment to only show INACTIVE for upper bodies in upper half
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cv2.putText(frame, "INACTIVE", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

    # Display the resulting frame
    cv2.imshow('Attendance System', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

**cap.read()** returns a bool (True/False). If frame is read correctly, it will be True. So you can check end of the video by checking this return value.
